# Домашка 6

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?

### Решение

Возьмем датасет из репозитория UCI. Датасет содержит данные о взрослых людях, размеченных на 2 класса: кто зарабатывает до 50K в год и больше 50К в год.

Описание данных - https://archive.ics.uci.edu/ml/datasets/Adult

In [92]:
import pandas as pd
import numpy as np
data = pd.read_csv("adult.data")
print(data.shape)
data.head(2)

(32560, 15)


,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [93]:
# переименуем и преобразуем целевую колонку
cols = data.columns.tolist()
cols[-1] = 'more_50K'
data.columns = cols

data.loc[data['more_50K']==' <=50K', 'more_50K'] = 0
data.loc[data['more_50K']==' >50K', 'more_50K'] = 1
data['more_50K'] = data['more_50K'].astype('int')

In [94]:
cols = data.columns
cols_new = []
for col in cols:
    cols_new.append(str(col).strip())
    
data.columns = cols_new

In [95]:
print(data['more_50K'].value_counts(normalize=True))
data.tail(3)

0    0.759183
1    0.240817
Name: more_50K, dtype: float64


,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,more_50K
32557,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32558,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0
32559,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,1


In [96]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((26048, 14), (6512, 14), (26048,), (6512,))

In [97]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   39             32560 non-null  int64 
 1   State-gov      32560 non-null  object
 2   77516          32560 non-null  int64 
 3   Bachelors      32560 non-null  object
 4   13             32560 non-null  int64 
 5   Never-married  32560 non-null  object
 6   Adm-clerical   32560 non-null  object
 7   Not-in-family  32560 non-null  object
 8   White          32560 non-null  object
 9   Male           32560 non-null  object
 10  2174           32560 non-null  int64 
 11  0              32560 non-null  int64 
 12  40             32560 non-null  int64 
 13  United-States  32560 non-null  object
 14  more_50K       32560 non-null  int32 
dtypes: int32(1), int64(6), object(8)
memory usage: 3.6+ MB


In [112]:
import catboost as ctb
cat_feats = ['State-gov', 'Bachelors', 'Never-married', 'Adm-clerical', 'Not-in-family', 'White', 'Male', 'United-States']

model = ctb.CatBoostClassifier(cat_features=cat_feats, random_state=123, verbose=False)

In [113]:
%%time

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

Wall time: 1min 2s


In [123]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

results = []

def evaluate_results(y_test, y_predict, model_title: str, results: list):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    res = {'model': model_title, 'roc': roc, 'f1': f1, 'recall': rec, 'precision': prc}
    results.append(res)

    
evaluate_results(y_test, y_predict, 'classifier', results)

Classification results:
f1: 71.68%
roc: 80.13%
recall: 66.29%
precision: 78.02%


In [115]:
results

[{'model': 'classifier',
  'roc': 0.801275878548118,
  'f1': 0.7167630057803468,
  'recall': 0.6628930817610063,
  'precision': 0.7801628423390081}]

### Теперь попробуем PU Learning

In [118]:
def learn_pu_learning_model(data, positive_rate):
    
    mod_data = data.copy()
    #get the indices of the positives samples
    pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
    #shuffle them
    np.random.shuffle(pos_ind)
    # leave just 25% of the positives marked
    pos_sample_len = int(np.ceil(positive_rate * len(pos_ind)))
    print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
    pos_sample = pos_ind[:pos_sample_len]
    
    mod_data['class_test'] = -1
    mod_data.loc[pos_sample,'class_test'] = 1
    print('target variable:\n', mod_data.iloc[:,-1].value_counts())
    
    x_data = mod_data.iloc[:,:-2].values # just the X 
    y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
    y_positive = mod_data.iloc[:,-2].values # original class
    
    mod_data = mod_data.sample(frac=1)
    neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
    sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
    pos_sample = mod_data[mod_data['class_test']==1]
    print(neg_sample.shape, pos_sample.shape)
    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
    
    model = ctb.CatBoostClassifier(cat_features=cat_feats, random_state=123, verbose=False)
    
    model.fit(sample_train.iloc[:,:-2], 
    sample_train.iloc[:,-2])
    y_predict = model.predict(sample_test.iloc[:,:-2])
    
    evaluate_results(sample_test.iloc[:,-2], y_predict, 'PU_{}'.format(positive_rate), results)

In [124]:
%%time
learn_pu_learning_model(data, positive_rate=0.1)

Using 785/7841 as positives and unlabeling the rest
target variable:
 -1    31775
 1      785
Name: class_test, dtype: int64
(785, 16) (785, 16)
Classification results:
f1: 64.56%
roc: 82.21%
recall: 89.72%
precision: 50.42%
Wall time: 29.7 s


In [125]:
%%time
learn_pu_learning_model(data, positive_rate=0.25)

Using 1961/7841 as positives and unlabeling the rest
target variable:
 -1    30599
 1     1961
Name: class_test, dtype: int64
(1961, 16) (1961, 16)
Classification results:
f1: 60.71%
roc: 82.76%
recall: 91.80%
precision: 45.35%
Wall time: 32.9 s


In [126]:
%%time
learn_pu_learning_model(data, positive_rate=0.40)

Using 3137/7841 as positives and unlabeling the rest
target variable:
 -1    29423
 1     3137
Name: class_test, dtype: int64
(3137, 16) (3137, 16)
Classification results:
f1: 57.16%
roc: 83.14%
recall: 90.19%
precision: 41.84%
Wall time: 35.4 s


In [127]:
%%time
learn_pu_learning_model(data, positive_rate=0.55)

Using 4313/7841 as positives and unlabeling the rest
target variable:
 -1    28247
 1     4313
Name: class_test, dtype: int64
(4313, 16) (4313, 16)
Classification results:
f1: 52.63%
roc: 84.23%
recall: 90.56%
precision: 37.09%
Wall time: 41.6 s


In [128]:
%%time
learn_pu_learning_model(data, positive_rate=0.70)

Using 5489/7841 as positives and unlabeling the rest
target variable:
 -1    27071
 1     5489
Name: class_test, dtype: int64
(5489, 16) (5489, 16)
Classification results:
f1: 43.82%
roc: 84.13%
recall: 89.07%
precision: 29.06%
Wall time: 1min


In [132]:
%%time
learn_pu_learning_model(data, positive_rate=0.85)

Using 6665/7841 as positives and unlabeling the rest
target variable:
 -1    25895
 1     6665
Name: class_test, dtype: int64
(6665, 16) (6665, 16)
Classification results:
f1: 28.94%
roc: 85.06%
recall: 90.11%
precision: 17.24%
Wall time: 47.7 s


In [134]:
%%time
learn_pu_learning_model(data, positive_rate=0.99)

Using 7763/7841 as positives and unlabeling the rest
target variable:
 -1    24797
 1     7763
Name: class_test, dtype: int64
(7763, 16) (7763, 16)
Classification results:
f1: 3.15%
roc: 84.08%
recall: 86.21%
precision: 1.61%
Wall time: 49.1 s


In [136]:
%%time
learn_pu_learning_model(data, positive_rate=0.05)

Using 393/7841 as positives and unlabeling the rest
target variable:
 -1    32167
 1      393
Name: class_test, dtype: int64
(393, 16) (393, 16)
Classification results:
f1: 62.39%
roc: 80.48%
recall: 91.84%
precision: 47.24%
Wall time: 36.7 s


In [140]:
pd.set_option('precision', 2)
table = pd.DataFrame.from_records(results)
table.sort_values(by=['model'], ascending=True, inplace=True)
table

,model,roc,f1,recall,precision
8,PU_0.05,0.80,0.62,0.92,0.47
1,PU_0.1,0.82,0.65,0.90,0.50
2,PU_0.25,0.83,0.61,0.92,0.45
3,PU_0.4,0.83,0.57,0.90,0.42
4,PU_0.55,0.84,0.53,0.91,0.37
5,PU_0.7,0.84,0.44,0.89,0.29
6,PU_0.85,0.85,0.29,0.90,0.17
7,PU_0.99,0.84,0.03,0.86,0.02
0,classifier,0.80,0.72,0.66,0.78


### Выводы

Получились результаты, которые я не могу однозначно объяснить, но попробую.

1.Roc_auc почти не зависит от количества позитивных примеров и совпадает классическим классификатором. Recall PU learning тоже практически не зависит от количества позитивных примеров. Хотя существенно превосходит классический классификатор. Precision имеет максимальное значение при определенной доле позитивов. В моем случае это 10% от всех позитивных ответов. Дальше precision начинает падать. Причем после увеличения доли позитивов до 50% снижение очень резкое.

2.Мне кажется, что при использовании PU learning мы получаем сильно смещенный оптимальный порог вероятности. Его обязательно надо определять. Иначе я никак не могу объяснить, что когда доля позитивов стремиться к 100%, так сильно падает precision, при этом roc_auc остается таким же.

3.В целом результаты использования PU learning при даже небольшом количестве известных позитивов (от 10% до 25%) дает вполне неплохой результат. И может использоваться как для классификации не до конца размеченной выборки, так и для решения задачи lookalike. Правда, не исключаю, что результаты на других датасетах могут сильно отличаться от текущих.